In [1]:
!pip install --upgrade -q google-colab
!apt-get install -q gifsicle
!pip install -q pygifsicle
# Yes, three different image libraries: 
#  PIL for drawing, imageio for gif creation, gifsicle for gif optimization
import imageio
from PIL import Image, ImageDraw
format='GIF-PIL'
# Format - Pillow is the default!
F = imageio.formats["GIF"]
assert F.name == "GIF-PIL"

from pygifsicle import optimize
import gc

import scipy.constants
import os

import math
import numpy as np

Reading package lists...
Building dependency tree...
Reading state information...
gifsicle is already the newest version (1.91-2).
The following package was automatically installed and is no longer required:
  libnvidia-common-430
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 25 not upgraded.


In [2]:
%cd /content
!rm -rf remap_image
!git clone -q https://github.com/LanceNorskog/remap_image.git
%cd /content/remap_image
import scurve.scurve as scurve

side = 8
side = 8
tiling = 'hilbert'
map = scurve.fromSize(tiling, 2, side * side)
def get_reverse_index(index, side):
    xy = map.point(index)
    return xy[0] * side + xy[1]

def remap(images, inverted=False):
    side = images.shape[-1]
            
    h_indexes = np.arange(0, (side * side), dtype='int32')
    h_indexes_inverted = np.arange(0, (side * side), dtype='int32')
    for index in range(side * side):
        h_indexes[index] = get_reverse_index(index, side)
    for x in range(side):
        for y in range(side):
            h_indexes_inverted[x * side + y] = map.index([x, y])        
    tiles = np.zeros((images.shape[0], side, side))
    for i in range(images.shape[0]):
        padded = np.reshape(np.pad(images[i], (0, side-side), mode='constant', constant_values=(0)), (side*side))
        if not inverted:
            tiles[i] = np.reshape(padded[h_indexes], (side, side))
        else:
            tiles[i] = np.reshape(padded[h_indexes_inverted], (side, side))
    return tiles

/content
/content/remap_image
[8, 8]


In [0]:
# I thought numpy had an indentity matrix func but can't find it!
def identity(mat):
    indexes = np.arange(0, mat.shape[1])
    mat[indexes, indexes] = 1
idmat = np.zeros((64, 64), dtype='float32')
identity(idmat)
ident_images = np.reshape(idmat, (64, 8, 8))

In [0]:
# composite current and N previous images into one image
def sum_images(images, span=8):
    num_frames = images.shape[0]-(span-1)
    summed = np.zeros((num_frames, 8, 8), dtype='float32')
    for i in range(num_frames):
        for j in range(span):
            summed[i] += images[i + j]
        gray = np.max(summed[i])
        if not math.isnan(gray):
            summed[i] = summed[i] / gray
    return summed

# same as above but also include set of previous summations as faint echos
def ghost_images(images, ghosts=3, shade=0.3):
    num_frames = images.shape[0]
    ghosted = np.zeros((num_frames, 8, 8), dtype='float32')
    for i in range(0, num_frames):
        for ghost in range(ghosts, -1, -1):
            if i-ghost >= 0:
                for r in range(images.shape[1]):
                    for c in range(images.shape[2]):
                        pix = images[i-ghost][r][c]
                        if pix > 0.001:
                            ghosted[i][r][c] = max(pix - (ghost * shade), 0)
    return ghosted

# add N copies of first and last images as lead-in and tail images 
def head_tail(frames, head, tail):
    out = [frames[0] for x in range(head)]
    for x in range(len(frames)):
        out.append(frames[x])
    for x in range(tail):
        out.append(frames[-1])
    return np.asarray(out, dtype='uint8')

def get_centroid(frame, value=0.5):
    side = frame.shape[0]
    points = []
    for r in range(side):
        for c in range(side):
            if frame[r][c] > value:
                points.append([r,c])
    if len(points) == 0:
        return []
    data = np.transpose(np.array(points))
    return [np.mean(data[0]), np.mean(data[1])]

def get_centroids(frames):
    return np.array([get_centroid(frame) for frame in frames])


In [5]:
box_size = 16
gap = int(((side + 1) * box_size)/scipy.constants.golden)
head = 5
tail = 8
spans = 3

summed_ident = sum_images(ident_images)
summed_hilbert = sum_images(remap(ident_images, inverted=True))


num_frames: 57
num_frames: 57


In [0]:
# render checkerboard grid
def grid(side, frame, grid_color):
    # horz
    for r in range(side + 1):
        for c in range(side * box_size):
            frame[r * box_size][c] = grid_color
    # vert
    for r in range(side * box_size + 1):
        for c in range(side+ 1):
            frame[r][c * box_size] = grid_color

# expand image
def blowup(image, size):
    im_obj = Image.fromarray(image)
    big = im_obj.resize((size, size))
    return np.copy(np.asarray(big))
 
# paste up two same-size frames side by side with a gap
def dual_anim(image, image2, gap):
    edge = image.shape[0]
    shape = (image.shape[0], image.shape[1]*2 + gap)
    frame = np.zeros(shape, dtype='uint8')
    for r in range(edge):
        frame[r][0:edge] = image[r][0:edge + 1]
        frame[r][-(edge):] = image2[r][0:edge + 1]
    return frame

# turn N 8-bit numpy matrices into gif frames, movie, optimize it (1mb -> 50k)
def animate(images, movie_file):
    imageio.mimwrite(movie_file, images, fps=2)
    optimize(movie_file)


In [0]:
# draw dots and lines on rendered frame
def scribble_dot(frame, centroid):
    row = int(round(centroid[0]))
    col = int(round(centroid[1]))
    for r in [-2, -1, 0, 1, 2]:
        for c in [-2, -1, 0, 1, 2]:
            if abs(r) + abs(c) < 4:
                frame[row + r][col + c] = 255

def scribble_line(frame, indx, centroids):
    def draw_line(frame, p1, p2):
        image = Image.fromarray(frame)
        draw = ImageDraw.Draw(image) 
        draw.line((p1[0], p1[1], p2[0], p2[1]), fill=255, width=2)
        for r in range(frame.shape[0]):
            for c in range(frame.shape[1]):
                frame[r][c] = image.getpixel((c,r))
    for i in range(indx):
        p1 = centroids[i]
        p2 = centroids[i+1]
        r1 = int(round(p1[0]))
        c1 = int(round(p1[1]))
        r2 = int(round(p2[0]))
        c2 = int(round(p2[1]))
        draw_line(frame, [c1, r1], [c2, r2])

In [0]:
filename='walking_hilbert.gif'
!rm -f walking_hilbert.gif

In [9]:
draw_ident = head_tail(summed_ident, head, tail)
draw_hilbert = head_tail(summed_hilbert, head, tail)

num_frames = draw_ident.shape[0]

centroids = get_centroids(draw_hilbert)
centroids = centroids * box_size + box_size//2

gif_frames = []
for i in range(num_frames):
    gc.collect()
    ident_large = blowup(draw_ident[i] * 192, box_size * side + 1)
    hilbert_large = blowup(draw_hilbert[i] * 192, box_size * side + 1)
    grid(side, ident_large, grid_color=252)
    grid(side, hilbert_large, grid_color=252)
    
    fr = dual_anim(ident_large, hilbert_large, gap=gap)
    gif_frames.append(fr)

animate(gif_frames, filename)


anim frame shape: (129, 346)


In [0]:
!sleep 5
from google.colab import files
files.download(filename)

In [0]:
filename='walking_hilbert_centroids.gif'
!rm -f walking_hilbert_centroids.gif

In [12]:
draw_ident = head_tail(summed_ident, head, tail)
draw_hilbert = head_tail(summed_hilbert, head, tail)

num_frames = draw_ident.shape[0]

centroids = get_centroids(draw_hilbert)
centroids = centroids * box_size + box_size//2

gif_frames = []
for i in range(num_frames):
    gc.collect()
    ident_large = blowup(draw_ident[i] * 192, box_size * side + 1)
    hilbert_large = blowup(draw_hilbert[i] * 192, box_size * side + 1)
    grid(side, ident_large, grid_color=252)
    grid(side, hilbert_large, grid_color=252)
    scribble_dot(hilbert_large, centroids[i])
    
    fr = dual_anim(ident_large, hilbert_large, gap=gap)
    gif_frames.append(fr)

animate(gif_frames, filename)


anim frame shape: (129, 346)


In [0]:
!sleep 5
from google.colab import files
files.download(filename)

In [14]:
filename='walking_hilbert_centroids_ghost.gif'
!rm -f walking_hilbert_centroids_ghost.gif
print(np.max(summed_hilbert))

1.0


In [15]:
prep_ident = head_tail(summed_ident, head, tail)
draw_hilbert = ghost_images(prep_ident, spans, shade=0.25)
prep_hilbert = head_tail(summed_hilbert, head, tail)
draw_hilbert = ghost_images(prep_hilbert, spans, shade=0.25)

num_frames = draw_ident.shape[0]

# do not get distracted by ghost cells, use prep_hilbert
centroids = get_centroids(prep_hilbert)
centroids = centroids * box_size + box_size//2

gif_frames = []
for i in range(num_frames):
    gc.collect()
    ident_large = blowup(draw_ident[i] * 192, box_size * side + 1)
    hilbert_large = blowup(draw_hilbert[i] * 192, box_size * side + 1)
    grid(side, ident_large, grid_color=252)
    grid(side, hilbert_large, grid_color=252)
    scribble_line(hilbert_large, i, centroids)
    scribble_dot(hilbert_large, centroids[i])
    
    fr = dual_anim(ident_large, hilbert_large, gap=gap)
    gif_frames.append(fr)

animate(gif_frames, filename)


anim frame shape: (129, 346)


In [0]:
!sleep 5
from google.colab import files
files.download(filename)